## 数据准备

In [2]:
library(tidyverse)
library(fastDummies)
all_data = read_csv("/Users/dddd1007/project2git/cognitive_control_model/data/input/all_data_with_RL_BL_estimate_result.csv")
all_data_without_error <- all_data %>%
                            filter(Type == "hit") %>%
                            filter(abs(RT - mean(RT)) < (sd(RT) * 3))
sub_num_list <- unique(all_data_without_error$Subject_num)
glimpse(all_data_without_error)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.1 ──

✔ ggplot2 3.3.5     ✔ purrr   0.3.4
✔ tibble  3.1.6     ✔ dplyr   1.0.7
✔ tidyr   1.1.4     ✔ stringr 1.4.0
✔ readr   2.1.1     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



ERROR: Error: '/Users/dddd1007/project2git/cognitive_control_model/data/input/all_data_with_bl_rl_model_full.csv' does not exist.


In [2]:
result_list <- list()
sub_count <- 1
for (i in sub_num_list) {
    tmp_data <- filter(all_data_without_error, Subject_num == i)
    mini_num <- 1
    mini_block <- vector(mode = "numeric", length = nrow(tmp_data))
    mini_block[1] <- 1
    for (line_num in 2:nrow(tmp_data)) {
        if (tmp_data$prop[line_num] != tmp_data$prop[line_num - 1]) {
            mini_num <- mini_num + 1
        }
        mini_block[line_num] <- mini_num
    }
    tmp_data$mini_block <- mini_block
    result_list[[sub_count]] <- tmp_data
    sub_count <- sub_count + 1
}
all_data_miniblock <- bind_rows(result_list, .id = "column_label")

## 估计概率与刺激特征之间的关系

In [1]:
# 和刺激特征的比较
aic_bl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab_v <- vector(mode = "numeric", length(sub_num_list))
aic_rl_sr   <- vector(mode = "numeric", length(sub_num_list))

for (i in seq_len(length(sub_num_list))) {
    single_sub_data <- all_data_without_error %>%
                        filter(Subject_num == sub_num_list[i])
    aic_bl_ab[i]   <- AIC(glm(congruency_num ~ bl_ab_r_con, data = single_sub_data, family = binomial))
    # aic_bl_sr[i]   <- AIC(glm(correct_action ~ bl_sr_r_left_hand, data = single_sub_data, family = binomial))
    aic_rl_ab[i]   <- AIC(glm(congruency_num ~ rl_ab_p, data = single_sub_data, family = binomial))
    aic_rl_ab_v[i] <- AIC(glm(congruency_num ~ rl_ab_v_p, data = single_sub_data, family = binomial))
    aic_rl_sr[i]   <- AIC(glm(correct_action ~ rl_sr_p, data = single_sub_data, family = binomial))
}

stim_aic <- data.frame(Subject_num = sub_num_list,
                     bl_ab = aic_bl_ab,
                     rl_ab = aic_rl_ab,
                     rl_ab_v = aic_rl_ab_v,
                     rl_sr = aic_rl_sr)

ERROR: Error in vector(mode = "numeric", length(sub_num_list)): object 'sub_num_list' not found


In [4]:
stim_aic

Subject_num,bl_ab,rl_ab,rl_ab_v,rl_sr
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,892.4169,1034.3374,1030.6187,1114.1005
2,921.8870,1055.3624,1054.7455,1122.7672
3,810.8296,920.3909,919.5475,973.7764
4,890.9563,1025.4604,1022.9266,1095.5766
5,930.2900,1061.4917,1059.4621,1126.8517
6,379.8674,430.4511,431.8100,427.0823
7,937.9951,1068.3894,1064.6194,1134.1653
8,794.9456,901.5649,898.0293,947.4803
9,916.8115,1056.8696,1050.7113,1126.9549


## 与反应时之间的变量关系

### 添加 Miniblock 信息的线性模型

In [26]:
rl_ab_result_AIC <- vector(
    mode = "numeric",
    length = length(sub_num_list)
)
rl_sr_result_AIC <- vector(
    mode = "numeric",
    length = length(sub_num_list)
)
rl_ab_v_result_AIC <- vector(
    mode = "numeric",
    length = length(sub_num_list)
)
bl_ab_result_AIC <- vector(
    mode = "numeric",
    length = length(sub_num_list)
)
bl_sr_result_AIC <- vector(
    mode = "numeric",
    length = length(sub_num_list)
)
for (i in seq_len(length(sub_num_list))) {
    tmp_data <- dummy_columns(
        filter(
            all_data_miniblock,
            Subject_num == sub_num_list[i]
        ),
        "mini_block"
    ) %>%
        select(-mini_block)

    bl_ab_data <- select(tmp_data, RT, bl_ab_r_selected, starts_with("mini_block"))
    model_bl_ab <- lm(RT ~ . -1, data = bl_ab_data)

    bl_sr_data <- select(tmp_data, RT, bl_sr_r_selected, starts_with("mini_block"))
    model_bl_sr <- lm(RT ~ . -1, data = bl_sr_data)

    rl_ab_data <- select(tmp_data, RT, rl_ab_p_selected, starts_with("mini_block"))
    model_rl_ab <- lm(RT ~ . -1, data = rl_ab_data)

    rl_sr_data <- select(tmp_data, RT, rl_sr_p_selected, starts_with("mini_block"))
    model_rl_sr <- lm(RT ~ . -1, data = rl_sr_data)

    rl_ab_v_data <- select(tmp_data, RT, rl_ab_v_p_selected, starts_with("mini_block"))
    model_rl_ab_v <- lm(RT ~ . -1, data = rl_ab_v_data)

    print(summary(lm(RT ~ ., data = select(tmp_data, RT, rl_ab_p_selected, congruency_num, Response, starts_with("mini_block")))))

    bl_ab_result_AIC[i] <- AIC(model_bl_ab)
    bl_sr_result_AIC[i] <- AIC(model_bl_sr)
    rl_ab_result_AIC[i] <- AIC(model_rl_ab)
    rl_sr_result_AIC[i] <- AIC(model_rl_sr)
    rl_ab_v_result_AIC[i] <- AIC(model_rl_ab_v)
}

# gather the result
AIC_result_table <- data.frame(
    bl_ab = bl_ab_result_AIC,
    bl_sr = bl_sr_result_AIC,
    rl_ab = rl_ab_result_AIC,
    rl_sr = rl_sr_result_AIC,
    rl_ab_v = rl_ab_result_AIC
)


Call:
lm(formula = RT ~ ., data = select(tmp_data, RT, rl_ab_p_selected, 
    congruency_num, Response, starts_with("mini_block")))

Residuals:
    Min      1Q  Median      3Q     Max 
-158.97  -56.41  -18.05   34.91  459.16 

Coefficients: (1 not defined because of singularities)
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)       499.457     12.430  40.182  < 2e-16 ***
rl_ab_p_selected  -83.678     10.794  -7.753 2.35e-14 ***
congruency_num     38.072      7.246   5.254 1.84e-07 ***
Response           -9.747      5.723  -1.703   0.0888 .  
mini_block_1      -15.352     16.703  -0.919   0.3583    
mini_block_2       26.581     16.270   1.634   0.1027    
mini_block_3       36.191     16.688   2.169   0.0304 *  
mini_block_4       16.794     12.798   1.312   0.1898    
mini_block_5       10.660     16.689   0.639   0.5231    
mini_block_6      -15.019     16.123  -0.932   0.3518    
mini_block_7        8.372     13.550   0.618   0.5368    
mini_block_8        8

In [27]:
AIC_result_table

bl_ab,bl_sr,rl_ab,rl_sr,rl_ab_v
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
11263.093,11268.683,11252.135,11262.105,11252.135
11137.827,11136.802,11105.870,11095.757,11105.870
10104.267,10104.347,10099.431,10094.409,10099.431
11250.797,11250.342,11245.516,11245.273,11245.516
11660.091,11661.472,11661.046,11666.556,11661.046
5164.954,5164.990,5164.837,5164.879,5164.837
11334.608,11334.315,11329.867,11331.132,11329.867
9760.915,9767.623,9749.217,9759.954,9749.217
11204.293,11204.441,11196.448,11190.977,11196.448


### 不添加 miniblock 信息

In [9]:
# 根据反应时比较 AIC, 此时应当用当前s-r对应的p来拟合
aic_bl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_bl_sr   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_sr   <- vector(mode = "numeric", length(sub_num_list))
aic_rl_ab_v <- vector(mode = "numeric", length(sub_num_list))

for (i in seq_len(length(sub_num_list))) {
    single_sub_data <- all_data_without_error %>%
                        filter(Subject_num == sub_num_list[i])
    aic_bl_ab[i]   <- AIC(lm(RT ~ bl_ab_r_selected + congruency_num + Response - 1, data = single_sub_data))
    print(summary(lm(RT ~ bl_ab_r_selected + congruency_num + Response - 1, data = single_sub_data)))
    aic_bl_sr[i]   <- AIC(lm(RT ~ bl_sr_r_selected + congruency_num + Response - 1, data = single_sub_data))
    aic_rl_ab[i]   <- AIC(lm(RT ~ rl_ab_p_selected + congruency_num + Response - 1, data = single_sub_data))
    aic_rl_sr[i]   <- AIC(lm(RT ~ rl_sr_p_selected + congruency_num + Response - 1, data = single_sub_data))
    aic_rl_ab_v[i] <- AIC(lm(RT ~ rl_ab_v_p_selected + congruency_num + Response - 1, data = single_sub_data))
}

RT_aic <- data.frame(Subject_num = sub_num_list,
                     bl_ab = aic_bl_ab,
                     bl_sr = aic_bl_sr,
                     rl_ab = aic_rl_ab,
                     rl_sr = aic_rl_sr,
                     rl_ab_v = aic_rl_ab_v)

RT_aic


Call:
lm(formula = RT ~ bl_ab_r_selected + rl_ab_p_selected + congruency_num + 
    Response - 1, data = single_sub_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-260.41  -75.88   -1.21  119.73  572.24 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
bl_ab_r_selected  982.612     43.026  22.838  < 2e-16 ***
rl_ab_p_selected -434.233     38.919 -11.157  < 2e-16 ***
congruency_num     93.066      9.473   9.824  < 2e-16 ***
Response           47.416      9.456   5.014 6.35e-07 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 150 on 946 degrees of freedom
Multiple R-squared:  0.8957,	Adjusted R-squared:  0.8953 
F-statistic:  2031 on 4 and 946 DF,  p-value: < 2.2e-16


Call:
lm(formula = RT ~ bl_ab_r_selected + rl_ab_p_selected + congruency_num + 
    Response - 1, data = single_sub_data)

Residuals:
    Min      1Q  Median      3Q     Max 
-341.23  -74.72    6.98  109.93  545.95 

Coefficients:
        

Subject_num,bl_ab,bl_sr,rl_ab,rl_sr,rl_ab_v
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,12337.981,12230.129,12637.695,12624.890,12624.293
2,12109.038,12027.996,12418.773,12437.066,12416.015
3,10782.260,10719.668,11016.572,11006.719,11005.804
4,12110.148,12007.691,12381.398,12384.503,12380.552
5,12316.597,12239.817,12545.978,12507.273,12544.884
6,5518.439,5484.733,5628.933,5636.802,5626.542
7,12328.966,12239.475,12591.807,12599.603,12590.498
8,10816.492,10714.111,11085.664,11046.159,11087.674
9,12065.438,11973.902,12362.052,12364.194,12363.334


In [126]:
# 根据反应时比较 AIC, 此时应当用当前s-r对应的p来拟合
cor_bl_ab   <- vector(mode = "numeric", length(sub_num_list))
cor_bl_sr   <- vector(mode = "numeric", length(sub_num_list))
cor_rl_ab   <- vector(mode = "numeric", length(sub_num_list))
cor_rl_sr   <- vector(mode = "numeric", length(sub_num_list))
cor_rl_ab_v <- vector(mode = "numeric", length(sub_num_list))

for (i in seq_len(length(sub_num_list))) {
    single_sub_data <- all_data_without_error %>%
                        filter(Subject_num == sub_num_list[i])
    cor_bl_ab[i]   <- cor(single_sub_data$RT, single_sub_data$bl_ab_r_selected)
    cor_bl_sr[i]   <- cor(single_sub_data$RT, single_sub_data$bl_sr_r_selected)
    cor_rl_ab[i]   <- cor(single_sub_data$RT, single_sub_data$rl_ab_p_selected)
    cor_rl_sr[i]   <- cor(single_sub_data$RT, single_sub_data$rl_sr_p_selected)
    cor_rl_ab_v[i] <- cor(single_sub_data$RT, single_sub_data$rl_ab_v_p_selected)
}

RT_cor <- data.frame(Subject_num = sub_num_list,
                     bl_ab = cor_bl_ab,
                     bl_sr = cor_bl_sr,
                     rl_ab = cor_rl_ab,
                     rl_sr = cor_rl_sr,
                     rl_ab_v = cor_rl_ab_v)
RT_cor

Subject_num,bl_ab,bl_sr,rl_ab,rl_sr,rl_ab_v
<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,-0.21164618,-0.19938913,-0.23411889,-0.21506745,-0.2291984480
2,-0.17250686,-0.18426147,-0.23701781,-0.25517077,-0.2313299126
3,-0.10618801,-0.11282395,-0.13086605,-0.15724726,-0.1274164524
4,-0.11569469,-0.11232967,-0.13987695,-0.13625570,-0.1394258710
5,-0.11095066,-0.11203175,-0.10285310,-0.07025950,-0.0988539709
6,-0.02890597,-0.05903212,-0.00511966,-0.05540074,-0.0008507794
7,-0.19632312,-0.19325989,-0.20940085,-0.20603907,-0.2089842922
8,-0.18715656,-0.16151204,-0.22261705,-0.18610169,-0.2264825713
9,-0.09830946,-0.10303070,-0.13307100,-0.15779207,-0.1308097669


### 竞争性比较

In [8]:
# 竞争性比较
summary(lm(RT ~ bl_ab_r_selected + rl_ab_p_selected, data = all_data_without_error))


Call:
lm(formula = RT ~ bl_ab_r_selected + rl_ab_p_selected, data = all_data_without_error)

Residuals:
    Min      1Q  Median      3Q     Max 
-282.24  -89.41  -30.43   54.80  560.07 

Coefficients:
                 Estimate Std. Error t value Pr(>|t|)    
(Intercept)       537.544      2.843 189.057   <2e-16 ***
bl_ab_r_selected   -8.886      8.867  -1.002    0.316    
rl_ab_p_selected  -60.244      6.310  -9.547   <2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 130.6 on 31230 degrees of freedom
Multiple R-squared:  0.01712,	Adjusted R-squared:  0.01706 
F-statistic: 272.1 on 2 and 31230 DF,  p-value: < 2.2e-16
